In [ ]:
import tensorflow as tf
from urllib.request import urlopen
import numpy as np
import os
import time

In [ ]:
url = 'https://www.gutenberg.org/files/174/174.txt'
text = urlopen(url).read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

print('{')
for char,_ in zip(char2idx, range(50)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')


Length of text: 462103 characters
85 unique characters
{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  '4' :  20,
  '5' :  21,
  '6' :  22,
  '7' :  23,
  '8' :  24,
  '9' :  25,
  ':' :  26,
  ';' :  27,
  '?' :  28,
  '@' :  29,
  'A' :  30,
  'B' :  31,
  'C' :  32,
  'D' :  33,
  'E' :  34,
  'F' :  35,
  'G' :  36,
  'H' :  37,
  'I' :  38,
  'J' :  39,
  'K' :  40,
  'L' :  41,
  'M' :  42,
  'N' :  43,
  'O' :  44,
  'P' :  45,
  'Q' :  46,
  'R' :  47,
  'S' :  48,
  'T' :  49,
  ...
}


In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

T
h
e
 
P
'The Project Gutenberg EBook of The Picture of Dorian Gray, by Oscar Wilde\r\n\r\nThis eBook is for the us'
'e of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give i'
't away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or on'
'line at www.gutenberg.net\r\n\r\n\r\nTitle: The Picture of Dorian Gray\r\n\r\nAuthor: Oscar Wilde\r\n\r\nRelease Da'
'te: June 9, 2008 [EBook #174]\r\n[This file last updated on July 2, 2011]\r\n[This file last updated on J'


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 85) # (batch_size, sequence_length, vocab_size)


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'handed to his master.\r\n"Her Grace told me to wait for an answer," he murmured.\r\n\r\nDorian put the let'

Next Char Predictions: 
 'Xg:e:#zn;2?Q"smb\r2@G\re5rgVPVNOX]093 -N_B)s_YP;.a64!%)tTtV:fBYw4?F\'k?nnHxE)z7,:xumO1/@P/srEgoUZfxebn@'


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4442515


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=35
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/35
71/71 [==============================] - 3s 42ms/step - loss: 3.1583
Epoch 2/35
71/71 [==============================] - 3s 42ms/step - loss: 2.2881
Epoch 3/35
71/71 [==============================] - 3s 42ms/step - loss: 2.1007
Epoch 4/35
71/71 [==============================] - 3s 42ms/step - loss: 1.9211
Epoch 5/35
71/71 [==============================] - 3s 42ms/step - loss: 1.7729
Epoch 6/35
71/71 [==============================] - 3s 42ms/step - loss: 1.6453
Epoch 7/35
71/71 [==============================] - 3s 42ms/step - loss: 1.5414
Epoch 8/35
71/71 [==============================] - 3s 42ms/step - loss: 1.4581
Epoch 9/35
71/71 [==============================] - 3s 42ms/step - loss: 1.3905
Epoch 10/35
71/71 [==============================] - 3s 42ms/step - loss: 1.3342
Epoch 11/35
71/71 [==============================] - 3s 42ms/step - loss: 1.2836
Epoch 12/35
71/71 [==============================] - 3s 42ms/step - loss: 1.2385
Epoch 13/35
71/71 [==================

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_35'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            21760     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 85)             87125     
Total params: 4,047,189
Trainable params: 4,047,189
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"For"))

Ford Henry, like
the day.  The reason who it was to
tea-table.  Herever e missed
by his sicts experience.
He had marred him.

Turned his spoke and stood looking of a coupt that had mould have to speak to him, some nd my whole question."

The elder woman who I am
to forget you, Harry.  It is not my words:
Shry afraid of himself.  But the world might go torrs.  Royal, he flum his hat and cape on the table, and Pattis work at dinner
name and in front of the
purpher and satisfife.  What is the number of your life.  Keep your horrible lend.  I met his a meat-seathe one cheeks, and anyoked about the sentence,
sometimes with a deep note of a certain
Vone.  It would be a real playinger with red gulters
became diarary of chiefly of pleasures, or alone with the top of the guns?  He bettered now.  A
tridus villages that seemed to end his
dreadful, frong the cabinet woman's
lucky from paradoxes, and that

Did it mnem like their secret.  When they met and watched the flaring stips.


In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

INFO:tensorflow:Assets written to: saved_model/my_model/assets
